In [1]:
from google.colab import drive
drive.mount('/content/drive')

ValueError: mount failed

In [2]:
base_folder = "/content/drive/MyDrive/Colab Notebooks/housing_fall2025"
%cd "{base_folder}"

/content/drive/MyDrive/Colab Notebooks/housing_fall2025


In [3]:
import sqlite3
import pandas as pd
conn = sqlite3.connect(f"{base_folder}/data/housing.db")
housing = pd.read_sql_query(
    """
    SELECT
        b.block_id,
        b.longitude,
        b.latitude,
        s.housing_median_age,
        s.total_rooms,
        s.total_bedrooms,
        s.population,
        s.households,
        s.median_income,
        s.median_house_value,
        op.name AS ocean_proximity
    FROM block AS b
    JOIN block_housing_stats AS s
        ON s.block_id = b.block_id
    JOIN ocean_proximity AS op
        ON op.ocean_proximity_id = b.ocean_proximity_id
    ORDER BY b.block_id
    """,
    conn,
)
conn.close()

housing.head()

,block_id,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,0,-122.23,37.88,41.0,880,129.0,322,126,8.3252,452600.0,NEAR BAY
1,1,-122.22,37.86,21.0,7099,1106.0,2401,1138,8.3014,358500.0,NEAR BAY
2,2,-122.24,37.85,52.0,1467,190.0,496,177,7.2574,352100.0,NEAR BAY
3,3,-122.25,37.85,52.0,1274,235.0,558,219,5.6431,341300.0,NEAR BAY
4,4,-122.25,37.85,52.0,1627,280.0,565,259,3.8462,342200.0,NEAR BAY


In [1]:
# notebooks/04_generate_streamlit_options.ipynb

import sys
import os
import sqlite3
import pandas as pd
import json
from pathlib import Path

# -----------------------------------------------------------------------------
# 1. ROBUST SETUP
# -----------------------------------------------------------------------------
current_dir = os.getcwd()

# Find the database
db_paths = [
    os.path.join(current_dir, '..', 'data', 'churn.db'),
    os.path.join(current_dir, 'data', 'churn.db')
]

db_path = None
for path in db_paths:
    if os.path.exists(path):
        db_path = path
        break

if not db_path:
    raise FileNotFoundError("❌ Could not find churn.db")

print(f"✓ Database found at: {db_path}")

# -----------------------------------------------------------------------------
# 2. GENERATE SCHEMA
# -----------------------------------------------------------------------------
conn = sqlite3.connect(db_path)

# Load data to analyze ranges/options
query = """
SELECT c.CreditScore, g.Name AS Geography, gen.Name AS Gender, c.Age, c.Tenure, c.Balance, 
       c.NumOfProducts, c.HasCrCard, c.IsActiveMember, c.EstimatedSalary
FROM customer c 
JOIN geography g ON c.GeographyID = g.GeographyID 
JOIN gender gen ON c.GenderID = gen.GenderID
"""
df = pd.read_sql_query(query, conn)
conn.close()

schema = {
    "numerical": {},
    "categorical": {}
}

# --- Analyze Numerical Features (for Sliders) ---
numerical_cols = ["CreditScore", "Age", "Tenure", "Balance", "NumOfProducts", "EstimatedSalary"]

for col in numerical_cols:
    schema["numerical"][col] = {
        "min": float(df[col].min()),
        "max": float(df[col].max()),
        "mean": float(df[col].mean()),
        "median": float(df[col].median())
    }

# --- Analyze Categorical Features (for Dropdowns) ---
categorical_cols = ["Geography", "Gender", "HasCrCard", "IsActiveMember"]

for col in categorical_cols:
    # Get unique values and sort them
    unique_vals = sorted(df[col].unique().tolist())
    schema["categorical"][col] = {
        "unique_values": unique_vals
    }

# -----------------------------------------------------------------------------
# 3. SAVE TO JSON
# -----------------------------------------------------------------------------
# Find the 'data' folder
data_dir = os.path.dirname(db_path)
output_file = os.path.join(data_dir, "data_schema.json")

with open(output_file, 'w') as f:
    json.dump(schema, f, indent=2)

print("\n" + "="*60)
print(f"✓ Schema saved to: {output_file}")
print("="*60)
print(json.dumps(schema, indent=2))

✓ Database found at: /Users/nayanpaliwal/Desktop/Eas_final_project/housing_app_fall25/data/churn.db

✓ Schema saved to: /Users/nayanpaliwal/Desktop/Eas_final_project/housing_app_fall25/data/data_schema.json
{
  "numerical": {
    "CreditScore": {
      "min": 360.0,
      "max": 848.0,
      "mean": 602.13,
      "median": 600.0
    },
    "Age": {
      "min": 18.0,
      "max": 89.0,
      "mean": 50.44,
      "median": 48.5
    },
    "Tenure": {
      "min": 0.0,
      "max": 9.0,
      "mean": 4.65,
      "median": 5.0
    },
    "Balance": {
      "min": 1513.8873195444046,
      "max": 198998.40359781377,
      "mean": 107636.16434825516,
      "median": 105285.99870514328
    },
    "NumOfProducts": {
      "min": 1.0,
      "max": 4.0,
      "mean": 2.24,
      "median": 2.0
    },
    "EstimatedSalary": {
      "min": 20231.907926596057,
      "max": 149672.63691424497,
      "mean": 77630.49664295535,
      "median": 69482.64419393308
    }
  },
  "categorical": {
    "Geogr